## Mapping The 100 Best Coffee Shops For the Top 5 Cities for Coffee in the US

### Objective

#### Many coffee enthusiats often travel just to experience the best coffee that a city can deliver.  According to CNBC.com the top 5 coffee cities in the US are the following: . 
###### New York, NY, Seattle, WA, San Francisco, CA, Portland, OR, and Los Angeles, CA

#### Pulling location data from Foursquare's api I would like to map out the top 100 coffee locations so that a coffee enthusiats can plan their trips with these maps in mind.  For example rather than staying at a hotel just because it has a good rating 



In [1]:

import foursquare as fs
import pandas as pd
import numpy as np

import requests

from pandas.io.json import json_normalize
import folium


In [2]:
CLIENT_ID = "3LLWO3UBNHUOG1GN4ED1ATZNTCFJSMUWZOS4V3DXMLE0J1SF" # your Foursquare ID
CLIENT_SECRET = "UWBETEEF351CSDEJIAZHNY2LRIEWIZJLQZUOJ0UIIFE2AL4L"
VERSION = '20180604'
LIMIT = 100

#### Define top 5 cities to pull data from Foursquare

In [3]:

cities = ["Los Angeles, CA", 'Seattle, WA', 'San Francisco, CA', 'Portland, OR', 'New York, NY']

data = {}

for city in cities:
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1e0931735") # Coffee Shop CATEGORY ID
    
    data[city] = requests.get(url).json()

#### Preview Queried Data

In [4]:
data[city]['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '56292858498e5a271a288948',
   'name': 'té (Té Company)',
   'location': {'address': '163 W 10th St',
    'crossStreet': 'btwn 7th Ave S & Waverly Pl',
    'lat': 40.734537261775436,
    'lng': -74.00182753801346,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.734537261775436,
      'lng': -74.00182753801346}],
    'postalCode': '10014',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['163 W 10th St (btwn 7th Ave S & Waverly Pl)',
     'New York, NY 10014',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1dc931735',
     'name': 'Tea Room',
     'pluralName': 'Tea Rooms',
     'shortName': 'Tea Room',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_',
      'suffix': '.png'},
     'primary': True}]

#### Create a dictionary of all the top 100 coffee venues for each of the four cities


In [5]:

venues_dict ={}

#itterate through the array of cities, 

for city in cities:
    venues = json_normalize(data[city]['response']['groups'][0]['items'])
    venues_dict[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    # Name the columns to their respective data
    venues_dict[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [6]:
# pd.Dataframe(df_venues)
type(venues_dict)


dict

#### create a dictionary of the lat and long points for each coffee venue


In [7]:
maps = {}

for city in cities:
    city_lat = np.mean([data[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        data[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([data[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        data[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    
    
    # add markers to map
    for lat, lng, label in zip(venues_dict[city]['Lat'], venues_dict[city]['Lng'], venues_dict[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Coffee Shops in {city} = ", data[city]['response']['totalResults'])
    print(f"\n")

Total number of Coffee Shops in Los Angeles, CA =  206


Total number of Coffee Shops in Seattle, WA =  202


Total number of Coffee Shops in San Francisco, CA =  212


Total number of Coffee Shops in Portland, OR =  195


Total number of Coffee Shops in New York, NY =  212




#### Map of Los Angeles California

In [8]:
maps[cities[0]]

#### Map of Seattle Washington

In [9]:
maps[cities[1]]

#### Map of San Francisco California

In [10]:
maps[cities[2]]

#### Map of Portland Oregon

In [11]:
maps[cities[3]]

#### Map of New York New York

In [12]:
maps[cities[4]]